In [1]:
import time
import pandas as pd
from census import Census

# Census API access
api_key = "639f2aedf7c17b164527591258cda00b25249b4b"
c = Census(key=api_key)

# Conditions

What are the conditions of the census tracts within Richmond City, North Richmond, and Rollingwood, California, in 2022 and 2017?

## Race
What is the racial and ethnic percentages of each census tract? Are there any census tracts with MOEs that are too high to include?

In [12]:
def process_acs_df(in_df):
    '''This is a *docstring*. It's a brief explanation of what the function does,
    and what inputs and outputs it has.

    Inputs:
    - in_df (pd.DataFrame): a DataFrame containing race/ethnicity data from Table B03002

    Outputs:
    A modified version of in_df with certain races grouped and proportions and margins
    of error calculated
    '''
    
    # Start by making a copy of in_df, so we don't destroy the original data.
    # df is also a nice short name we can use throughout this function
    df = in_df.copy()

    ### CLEAN UNUSUAL MOES
    df = df.replace(-555555555.0, 0)

    ### AGGREGATE ESTIMATES
    # Define a list of columns to combine into "nh_other"
    nh_other_cols = ['nh_native', 'nh_pi', 'nh_1other', 'nh_multi']

    # Calculate nh_other
    df['nh_other'] = df[nh_other_cols].sum(axis='columns')

    # Use a list comprehension to append "_moe" to all strings in our list
    nh_other_moes = [f'{col}_moe' for col in nh_other_cols]

    # Use this new list to calculate nh_other_moe    
    df['nh_other_moe'] = (df[nh_other_moes]**2).sum(axis='columns')**0.5

    ### CALCULATE PROPORTIONS
    # Iterate through the racial/ethnic groups we're using
    for group in ['nh_white', 'nh_black', 'nh_asian', 'nh_other', 'hispanic']:
        # Calculate the proportion for this group
        df[f'pct_{group}'] = df[group] / df['total']
    
        # Calculate the MOE for this proportion
        df[f'pct_{group}_moe'] = (df[f'{group}_moe']**2 - df[f'pct_{group}']**2 * df['total_moe']**2)**0.5 / df['total']

    return df  # Don't forget to *return* the modified DataFrame

In [17]:
# Define the dict of variables to pull and rename
variables_of_interest = {
    'NAME': 'NAME',
    'GEO_ID': 'GEO_ID',
    'B03002_001E': 'total',
    'B03002_001M': 'total_moe',
    'B03002_003E': 'nh_white',
    'B03002_003M': 'nh_white_moe',
    'B03002_004E': 'nh_black',
    'B03002_004M': 'nh_black_moe',
    'B03002_005E': 'nh_native',
    'B03002_005M': 'nh_native_moe',
    'B03002_006E': 'nh_asian',
    'B03002_006M': 'nh_asian_moe',
    'B03002_007E': 'nh_pi',
    'B03002_007M': 'nh_pi_moe',
    'B03002_008E': 'nh_1other',
    'B03002_008M': 'nh_1other_moe',
    'B03002_009E': 'nh_multi',
    'B03002_009M': 'nh_multi_moe',
    'B03002_012E': 'hispanic',
    'B03002_012M': 'hispanic_moe',
}

# Get ACS 2022 5 year Table B03002 in Richmond City, North Richmond, Rollingwood, CA
# Get ACS 2022 5 year Table B03002 in Contra Costa County
# county:013
c_tracts = 'tract:380001,380002,378000,365002,365003,379000,377000,376000,374000,371000,367200'

df_tracts = pd.DataFrame(
    c.acs5.get(
        list(variables_of_interest.keys()),
        {'for': c_tracts, 'in': 'state:06 county:013'},
        year=2022
    )
)
df_county = pd.DataFrame(
    c.acs5.get(
        list(variables_of_interest.keys()),
        {'for': 'county:013', 'in': 'state:06'},
        year=2022
    )
)

# Rename the DataFrame columns again using the dict
df_tracts = df_tracts.rename(columns=variables_of_interest)
df_county = df_county.rename(columns=variables_of_interest)

# Combo races, generate percentages and percentage MOEs
df_tracts_processed = process_acs_df(df_tracts)
df_county_processed = process_acs_df(df_county)

#export
df_out = pd.concat([df_tracts_processed, df_county_processed])
df_out = df_out[[
    'NAME', 'GEO_ID',
    'pct_nh_white', 'pct_nh_white_moe',
    'pct_nh_black', 'pct_nh_black_moe',
    'pct_nh_asian', 'pct_nh_asian_moe',
    'pct_nh_other', 'pct_nh_other_moe',
    'pct_hispanic', 'pct_hispanic_moe',
]]
df_out

,NAME,GEO_ID,pct_nh_white,pct_nh_white_moe,pct_nh_black,pct_nh_black_moe,pct_nh_asian,pct_nh_asian_moe,pct_nh_other,pct_nh_other_moe,pct_hispanic,pct_hispanic_moe
0,Census Tract 3650.02; Contra Costa County; Cal...,1400000US06013365002,0.050474,0.031922,0.150648,0.053308,0.068265,0.036220,0.083349,0.052796,0.647264,0.085329
1,Census Tract 3650.03; Contra Costa County; Cal...,1400000US06013365003,0.191100,0.075867,0.290990,0.067845,0.160081,0.068008,0.117799,0.074564,0.240030,0.075972
2,Census Tract 3672; Contra Costa County; Califo...,1400000US06013367200,0.049753,0.015774,0.128642,0.055663,0.232748,0.068273,0.050775,0.027433,0.538081,0.088484
3,Census Tract 3710; Contra Costa County; Califo...,1400000US06013371000,0.267140,0.044124,0.102147,0.037705,0.153740,0.043517,0.083276,0.055140,0.393698,0.085460
4,Census Tract 3740; Contra Costa County; Califo...,1400000US06013374000,0.189460,0.040126,0.106038,0.029794,0.111372,0.055922,0.030937,0.022391,0.562193,0.118166
5,Census Tract 3760; Contra Costa County; Califo...,1400000US06013376000,0.006108,0.004680,0.132300,0.035578,0.042610,0.025595,0.050209,0.024538,0.768772,0.096441
6,Census Tract 3770; Contra Costa County; Califo...,1400000US06013377000,0.046767,0.025190,0.219780,0.083183,0.065040,0.035501,0.029773,0.025468,0.638640,0.131910
7,Census Tract 3780; Contra Costa County; Califo...,1400000US06013378000,0.655518,0.068601,0.010033,0.009159,0.079238,0.056336,0.071263,0.037889,0.183946,0.058905
8,Census Tract 3790; Contra Costa County; Califo...,1400000US06013379000,0.021114,0.008144,0.180007,0.037225,0.020637,0.019451,0.065848,0.035559,0.712394,0.090897
9,Census Tract 3800.01; Contra Costa County; Cal...,1400000US06013380001,0.412983,0.050460,0.187500,0.100655,0.200967,0.040790,0.075622,0.045880,0.122928,0.083597
